In [ ]:
from classes.hero import Hero

from scenarios.basic_example import friendly_minions, enemy_minions, hand_list
from scenarios.deck_example import deck_15, add_card_to_hand

from solver.solver import run_single_turn

from state.transition_state import start_turn, apply_results, end_turn, swap_roles


In [ ]:
def main():
    # 1) Create hero objects
    hero1 = Hero("Paladin", 30)
    hero2 = Hero("Warrior", 30)

    # 2) Define an inline function to build scenario data
    def build_scenario_data(friendly, enemy, hand, M=5, H_hero=12):
        m = len(friendly)
        n = len(enemy)
        h = len(hand)
        combined = friendly + hand
        A = [mn.attack for mn in combined]
        B = [mn.health for mn in combined]
        P = [enm.attack for enm in enemy]
        Q = [enm.health for enm in enemy]
        C = [mn.mana_cost for mn in hand]
        S = [mn.strat_value for mn in hand]
        return dict(m=m, n=n, h=h, M=M, H_hero=H_hero, A=A, B=B, P=P, Q=Q, C=C, S=S)

    # Let hero1 be "current_hero", hero2 be "opponent_hero" initially
    current_hero = hero1
    opponent_hero = hero2

    turn_number = 1

    # 3) Loop until one hero's health <= 0
    while hero1.health > 0 and hero2.health > 0:
        print(f"\n=== Turn {turn_number}: {current_hero.hero_class}'s turn ===")

        # Start turn (draw a card, etc.)
        start_turn(current_hero, deck_15, hand_list)

        # Build scenario data
        # - If current_hero is hero1, we treat friendly_minions as "friendly"
        #   and enemy_minions as "enemy". If it's hero2, we invert them.
        if current_hero == hero1:
            scenario_data = build_scenario_data(
                friendly_minions,
                enemy_minions,
                hand_list,
                M=5,
                H_hero=opponent_hero.health
            )
        else:
            scenario_data = build_scenario_data(
                enemy_minions,
                friendly_minions,
                hand_list,
                M=5,
                H_hero=opponent_hero.health
            )

        # Solve for this turn
        solution = run_single_turn(**scenario_data)

        # Apply the solver's results (like dealing damage)
        # Note: If your apply_results requires hero references, pass them
        apply_results(solution, scenario_data)

        # End turn checks (e.g. hero health)
        end_turn(current_hero, opponent_hero)
        if hero1.health <= 0 or hero2.health <= 0:
            break

        # Swap roles
        current_hero, opponent_hero = swap_roles(current_hero, opponent_hero)

        turn_number += 1

    # 4) After the loop, see who lost
    if hero1.health <= 0:
        print(f"\n{hero1.hero_class} hero lost. {hero2.hero_class} wins!")
    elif hero2.health <= 0:
        print(f"\n{hero2.hero_class} hero lost. {hero1.hero_class} wins!")
    else:
        print("\nGame ended for another reason.")

if __name__ == '__main__':
    main()





=== Turn 1: Paladin ===
Paladin draws: Minion: Gnomish Inventor | Class: Neutral | Attack: 2 | Health: 4 | Keywords: 
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 165U, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 42 rows, 48 columns and 126 nonzeros
Model fingerprint: 0x446514e4
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [3e-02, 4e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 25.0000000
Found heuristic solution: objective 42.0000000
Presolve removed 42 rows and 48 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Th

TypeError: apply_results() takes 2 positional arguments but 4 were given